In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = "https://raw.githubusercontent.com/fahmiagungmaulana/data1/main/train.csv"
train = pd.read_csv(url)

In [ ]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336


In [ ]:
train['id_titik_mulai'].nunique()

488

#Data Understanding

In [ ]:
train['kelompok'] = train.apply(lambda row: ''.join(sorted([str(row['id_titik_mulai']), str(row['id_titik_akhir'])])), axis=1)
# train['kelompok'] = train['kelompok'].astype(int)

In [ ]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,kelompok
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126,1425033102_21390008
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576,1677092762_579493410
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587,1930267566_26486694
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063,1111592522_3775231113
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336,5940503394_5940503398


In [ ]:
train['id_jalan'].nunique()

20

In [ ]:
train['id_titik_mulai'].nunique()

488

In [ ]:
train.kelompok.nunique()

In [ ]:
train['waktu_setempat'] = pd.to_datetime(train['waktu_setempat'])
train['tahun'] = train['waktu_setempat'].dt.year
train['bulan'] = train['waktu_setempat'].dt.month
train['tanggal'] = train['waktu_setempat'].dt.day
train['jam'] = train['waktu_setempat'].dt.strftime('%H:%M:%S')
train['zona'] = train['waktu_setempat'].dt.strftime('%z')

In [ ]:
train.zona.unique()

In [ ]:
train.tahun.unique()

In [ ]:
train.bulan.unique()

In [ ]:
train['jam'] = train['waktu_setempat'].dt.hour + train['waktu_setempat'].dt.minute / 60

In [ ]:
columns_to_drop = ['zona', 'tahun', 'bulan', 'waktu_setempat','id_titik_mulai', 'id_titik_akhir']
train.drop(columns=columns_to_drop, inplace=True)

In [ ]:
train.head()

#Test data

In [ ]:
url = "https://raw.githubusercontent.com/fahmiagungmaulana/data1/main/test.csv"
test = pd.read_csv(url)

In [ ]:
test['kelompok'] = test.apply(lambda row: ''.join(sorted([str(row['id_titik_mulai']), str(row['id_titik_akhir'])])), axis=1)

In [ ]:
test['waktu_setempat'] = pd.to_datetime(test['waktu_setempat'])
test['tahun'] = test['waktu_setempat'].dt.year
test['bulan'] = test['waktu_setempat'].dt.month
test['tanggal'] = test['waktu_setempat'].dt.day
test['jam'] = test['waktu_setempat'].dt.strftime('%H:%M:%S')
test['zona'] = test['waktu_setempat'].dt.strftime('%z')
test['jam'] = test['waktu_setempat'].dt.hour + test['waktu_setempat'].dt.minute / 60
columns_to_drop = ['zona', 'tahun', 'bulan', 'waktu_setempat','id_titik_mulai', 'id_titik_akhir']
test.drop(columns=columns_to_drop, inplace=True)

In [ ]:
test.head()

#Encoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
id_jalan_kelompok_train = train[['id_jalan', 'kelompok']]
encoded_train = encoder.fit_transform(id_jalan_kelompok_train)
id_jalan_kelompok_test =test[['id_jalan', 'kelompok']]
encoded_test = encoder.transform(id_jalan_kelompok_test)

In [ ]:
# Ubah hasil encoding menjadi DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=['encoded_id_jalan', 'encoded_kelompok'])
encoded_test_df = pd.DataFrame(encoded_test, columns=['encoded_id_jalan', 'encoded_kelompok'])

# Gabungkan hasil encoding dengan data train dan test
train_clean = pd.concat([train, encoded_train_df], axis=1)
test_clean = pd.concat([test, encoded_test_df], axis=1)


#Model

In [ ]:
X_train = train_clean.drop(['id_jalan', 'rerata_kecepatan', 'kelompok'], axis= 1)
X_test = test_clean.drop(['id_jalan', 'kelompok', 'id'], axis= 1)
y_train = train_clean['rerata_kecepatan']

In [ ]:
import xgboost as xgb

In [ ]:
SEED = 42
model = xgb.XGBRegressor(n_jobs=-1, random_state=SEED)
model.fit(X_train, y_train)

preds = model.predict(X_test)

In [ ]:
output = pd.DataFrame({'id': test_clean['id'],
                       'rerata_kecepatan': preds})

In [ ]:
output.head()

In [ ]:
output.shape

In [ ]:
test.shape

In [ ]:
output.to_csv('bismillah.csv', index=False)